# 類似度の計算
統計データは多くの場合テーブルデータになっている。ここで、テーブルデータの変数ごとに構成されているレコードをベクトルと考えることができ、ここから距離の計算を使用して類似度を算出することができる。ただし、注意する点として、ベクトルの各変数ごとに単位(尺度)が異なり、大きな数値に影響を受けやすい特徴がある。そこで、本章では類似度の計算と単位の影響および正規化や標準化をして単位を統一した場合の精度結果を確認する。

## ライブラリのインポート

In [1]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np

## データの読み込み

In [2]:
df = pd.read_csv("wine.csv")
df.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
1,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
2,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
4,1,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450


## 目的変数と説明変数の分離

In [3]:
y_name = "Wine"
y = df[y_name].values
x_tab = df.drop(y_name, axis=1)
x_name = x_tab.columns
x = x_tab.values

## 元々存在するデータと新しく確認するデータを分ける

In [4]:
x_train, x_test, y_train, y_test = tts(x, y, random_state=0, test_size=0.3)

## 類似度の算出
類似度の算出には「コサイン類似度」「ユークリッド距離」「マンハッタン距離」を使用する。

コサイン類似度は2つのベクトルの内積の数式について式変形を行いコサインの値を算出する。そのため1に近いと類似度が高く-1に近いと類似度が低い。

ユークリッド距離は2つのベクトルの直線距離を算出する。そのため、0に近いと類似度が高く値が大きいほど類似度が低い。

マンハッタン距離は2つのベクトルでベクトルごとの移動距離を算出する。そのため、0に近いと類似度が高く値が大きいと類似度が低い。

In [5]:
cos = []
euc = []
man = []
for i in range(len(x_test)):
    tmp_cos = []
    tmp_euc = []
    tmp_man = []
    for j in range(len(x_train)):
        tmp_cos.append([y_train[j], np.dot(x_test[i], x_train[j])/(np.linalg.norm(x_test[i])*np.linalg.norm(x_train[j]))])
        tmp_euc.append([y_train[j], np.linalg.norm(x_test[i]-x_train[j])])
        tmp_man.append([y_train[j], np.linalg.norm(x_test[i]-x_train[j], ord=1)])
    tmp_cos = sorted(tmp_cos, key=lambda x: x[1], reverse=True)
    tmp_euc = sorted(tmp_euc, key=lambda x: x[1])
    tmp_man = sorted(tmp_man, key=lambda x: x[1])
    cos.append(tmp_cos[0])
    euc.append(tmp_euc[0])
    man.append(tmp_man[0])

In [6]:
cos = np.array(cos)
euc = np.array(euc)
man = np.array(man)

In [7]:
y_cos = cos[:,0]
y_euc = euc[:,0]
y_man = man[:,0]

### 精度の確認

In [8]:
print("コサイン類似度")
print(classification_report(y_test, y_cos))
print("ユークリッド距離")
print(classification_report(y_test, y_euc))
print("マンハッタン距離")
print(classification_report(y_test, y_man))

コサイン類似度
              precision    recall  f1-score   support

           1       0.75      0.95      0.84        19
           2       0.90      0.90      0.90        21
           3       1.00      0.64      0.78        14

    accuracy                           0.85        54
   macro avg       0.88      0.83      0.84        54
weighted avg       0.88      0.85      0.85        54

ユークリッド距離
              precision    recall  f1-score   support

           1       0.85      0.89      0.87        19
           2       0.76      0.90      0.83        21
           3       0.56      0.36      0.43        14

    accuracy                           0.76        54
   macro avg       0.72      0.72      0.71        54
weighted avg       0.74      0.76      0.74        54

マンハッタン距離
              precision    recall  f1-score   support

           1       0.85      0.89      0.87        19
           2       0.90      0.86      0.88        21
           3       0.71      0.71      0.71      

基本的に精度は類似度の算出方法によってバラつきがあることが分かる。この理由としては大きい数値に影響を受けやすいことが挙げられる。そこで、次項では単位(尺度)を揃える。

## 最大最小正規化
最大最小正規化では変数の最大値を1として最小値を0にすることで全ての変数の単位(尺度)を揃える。

In [9]:
Mscale = MinMaxScaler()
Mscale.fit(x)
xm = Mscale.transform(x)

In [10]:
x_train, x_test, y_train, y_test = tts(xm, y, random_state=0, test_size=0.3)

In [11]:
cos = []
euc = []
man = []
for i in range(len(x_test)):
    tmp_cos = []
    tmp_euc = []
    tmp_man = []
    for j in range(len(x_train)):
        tmp_cos.append([y_train[j], np.dot(x_test[i], x_train[j])/(np.linalg.norm(x_test[i])*np.linalg.norm(x_train[j]))])
        tmp_euc.append([y_train[j], np.linalg.norm(x_test[i]-x_train[j])])
        tmp_man.append([y_train[j], np.linalg.norm(x_test[i]-x_train[j], ord=1)])
    tmp_cos = sorted(tmp_cos, key=lambda x: x[1], reverse=True)
    tmp_euc = sorted(tmp_euc, key=lambda x: x[1])
    tmp_man = sorted(tmp_man, key=lambda x: x[1])
    cos.append(tmp_cos[0])
    euc.append(tmp_euc[0])
    man.append(tmp_man[0])

In [12]:
cos = np.array(cos)
euc = np.array(euc)
man = np.array(man)

In [13]:
y_cos = cos[:,0]
y_euc = euc[:,0]
y_man = man[:,0]

In [14]:
print("コサイン類似度")
print(classification_report(y_test, y_cos))
print("ユークリッド距離")
print(classification_report(y_test, y_euc))
print("マンハッタン距離")
print(classification_report(y_test, y_man))

コサイン類似度
              precision    recall  f1-score   support

           1       0.86      1.00      0.93        19
           2       1.00      0.86      0.92        21
           3       1.00      1.00      1.00        14

    accuracy                           0.94        54
   macro avg       0.95      0.95      0.95        54
weighted avg       0.95      0.94      0.94        54

ユークリッド距離
              precision    recall  f1-score   support

           1       0.90      1.00      0.95        19
           2       1.00      0.86      0.92        21
           3       0.93      1.00      0.97        14

    accuracy                           0.94        54
   macro avg       0.95      0.95      0.95        54
weighted avg       0.95      0.94      0.94        54

マンハッタン距離
              precision    recall  f1-score   support

           1       0.90      1.00      0.95        19
           2       1.00      0.90      0.95        21
           3       1.00      1.00      1.00      

データの加工をする前と比較するとどの類似度計算を使用しても精度が90%以上に向上していることが分かる。

## 分散正規化(標準化)
単位(尺度)の揃え方として平均値を0として標準偏差を1にする。

In [15]:
Sscale = StandardScaler()
Sscale.fit(x)
xs = Sscale.transform(x)

In [16]:
x_train, x_test, y_train, y_test = tts(xs, y, random_state=0, test_size=0.3)

In [17]:
cos = []
euc = []
man = []
for i in range(len(x_test)):
    tmp_cos = []
    tmp_euc = []
    tmp_man = []
    for j in range(len(x_train)):
        tmp_cos.append([y_train[j], np.dot(x_test[i], x_train[j])/(np.linalg.norm(x_test[i])*np.linalg.norm(x_train[j]))])
        tmp_euc.append([y_train[j], np.linalg.norm(x_test[i]-x_train[j])])
        tmp_man.append([y_train[j], np.linalg.norm(x_test[i]-x_train[j], ord=1)])
    tmp_cos = sorted(tmp_cos, key=lambda x: x[1], reverse=True)
    tmp_euc = sorted(tmp_euc, key=lambda x: x[1])
    tmp_man = sorted(tmp_man, key=lambda x: x[1])
    cos.append(tmp_cos[0])
    euc.append(tmp_euc[0])
    man.append(tmp_man[0])

In [18]:
cos = np.array(cos)
euc = np.array(euc)
man = np.array(man)

In [19]:
y_cos = cos[:,0]
y_euc = euc[:,0]
y_man = man[:,0]

In [20]:
print("コサイン類似度")
print(classification_report(y_test, y_cos))
print("ユークリッド距離")
print(classification_report(y_test, y_euc))
print("マンハッタン距離")
print(classification_report(y_test, y_man))

コサイン類似度
              precision    recall  f1-score   support

           1       0.95      1.00      0.97        19
           2       1.00      0.90      0.95        21
           3       0.93      1.00      0.97        14

    accuracy                           0.96        54
   macro avg       0.96      0.97      0.96        54
weighted avg       0.97      0.96      0.96        54

ユークリッド距離
              precision    recall  f1-score   support

           1       0.90      1.00      0.95        19
           2       1.00      0.86      0.92        21
           3       0.93      1.00      0.97        14

    accuracy                           0.94        54
   macro avg       0.95      0.95      0.95        54
weighted avg       0.95      0.94      0.94        54

マンハッタン距離
              precision    recall  f1-score   support

           1       0.90      1.00      0.95        19
           2       1.00      0.90      0.95        21
           3       1.00      1.00      1.00      

単位を揃えたことでデータを加工する前と比べて精度が向上していることが分かる。また、コサイン類似度では精度が2%向上していることが分かる。